## Datasets for Dashboard

<p>Datasets required for creating visualisations in the dashboard. </p>

### Libraries to import

In [1]:
import pandas as pd
import os
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
from functions_file_3 import risk_country

In [2]:
directory = os.getcwd()
url = directory[:-7] + '2. Data/1. ECB Data/Portfolio Value Information/'
url_2 = directory[:-7] + '2. Data/1. ECB Data/'
path = directory[:-7] + '2. Data/4. Dashboard Data/'

### Functions to use

In [3]:
def clean_value(filename):
    # selecting rows for which we have downloaded ECB bonds data
    df = filename.iloc[12:,:]
    # calculating portfolio value for every quarter starting from June 2017 to Feb 2022
    df['DATE'] = df['End of Month'].map(lambda x: dt.datetime.strptime(x, '%b-%y'))
    df['DATE'] = df['End of Month'].map(lambda x: dt.datetime.strptime(x, '%b-%y'))
    df['Month'] = pd.to_datetime(df['DATE']).dt.strftime('%b')
    df['Year'] = pd.DatetimeIndex(df['DATE']).year
    df['Total holdings'] = df['Total holdings'].str.replace(",","")
    df['Total holdings'] = df['Total holdings'].apply(pd.to_numeric)
    columns_to_keep = ['Year','Month','Total holdings']
    data = df[columns_to_keep]
    return data

### Monthly CSPP holdings (June 2017 - Feb 2022)

In [4]:
portfolio_value = pd.read_csv(url + 'CSPP_breakdown_history.csv',skiprows=1,skipfooter=2, engine = 'python')

In [5]:
monthly_value = clean_value(portfolio_value)
monthly_value['Month_Number']=[dt.datetime.strptime(x, "%b") for x in monthly_value['Month']]
monthly_value['Month_Number'] = [x.month for x in monthly_value['Month_Number']]
monthly_value.head()

,Year,Month,Total holdings,Month_Number
12,2017,Jun,96620,6
13,2017,Jul,102226,7
14,2017,Aug,106898,8
15,2017,Sep,114658,9
16,2017,Oct,121607,10


In [6]:
monthly_value.to_csv(path + 'Monthly_Investments.csv')

### Sector wise 

In [7]:
yearly_sector_value = pd.read_excel(url_2 + 'Portfolio_Breakdown_Information.xlsx',sheet_name='Yearly_Sector_Breakdown')
yearly_portfolio = pd.read_excel(url_2 + 'Portfolio_Breakdown_Information.xlsx',sheet_name='Yearly_Portfolio_Value')


In [8]:
combined_data = yearly_sector_value.merge(yearly_portfolio,on = "YEAR", how ="left").drop_duplicates()
combined_data['SECTOR_HOLDINGS'] = (combined_data['SECTOR_CSPP_HOLDINGS']*combined_data['TOTAL_HOLDINGS(USD)']).round(0)
columns_to_keep = ['YEAR','ECONOMIC_SECTOR','SECTOR_HOLDINGS']
sector_data = combined_data[columns_to_keep]
sector_data.head()

,YEAR,ECONOMIC_SECTOR,SECTOR_HOLDINGS
0,2017,Automotive and parts,12036.0
1,2017,Beverages,6018.0
2,2017,Chemicals,6018.0
3,2017,Construction & Materials,7222.0
4,2017,Energy and basic resources,8425.0


### Country wise

In [9]:
yearly_country_value = pd.read_excel(url_2 + 'Portfolio_Breakdown_Information.xlsx',sheet_name='Yearly_Country_Breakdown')


In [10]:
combined_data = yearly_country_value.merge(yearly_portfolio,on = "YEAR", how ="left").drop_duplicates()
combined_data['COUNTRY_HOLDINGS'] = (combined_data['COUNTRY_CSPP_HOLDINGS']*combined_data['TOTAL_HOLDINGS(USD)']).round(0)
columns_to_keep = ['YEAR','RISK_COUNTRY','COUNTRY_HOLDINGS']
country_data = combined_data[columns_to_keep]
country_data.head()

,YEAR,RISK_COUNTRY,COUNTRY_HOLDINGS
0,2017,Belgium,6018.0
1,2017,France,34905.0
2,2017,Germany,30090.0
3,2017,Italy,13240.0
4,2017,Netherlands,7222.0


In [11]:
country_data.to_csv(path + 'Country_values.csv',index = None)
sector_data.to_csv(path + 'Sector_values.csv',index = None)

### Carbon Intensity for individual companies

In [12]:
CI_companies = pd.read_csv(path + 'CI_for_companies.csv')
master_file = pd.read_excel(directory[:-7] + '2. Data/Master_File.xlsx')

In [13]:
# count the number of bonds for every year
columns_to_keep = ['ISSUER_NAME','YEAR','ISIN_CODE']
file = master_file[columns_to_keep]
df = file.groupby(by = ['YEAR','ISSUER_NAME']).count()
df = df.reset_index()
columns = ['YEAR','ISSUER_NAME','NUMBER_OF_BONDS']
df.columns = columns

In [14]:
# combine with bonds info with carbon intensity of companies
df2 = df.merge(CI_companies, on = 'ISSUER_NAME',how = 'left').drop_duplicates()
df2 = df2.dropna()


In [15]:
carbon_intensity_companies = df2.sort_values(by= 'CARBON_INTENSITY',ascending = False)
carbon_intensity_companies.head()

,YEAR,ISSUER_NAME,NUMBER_OF_BONDS,TOTAL__REVENUE,ECONOMIC_SECTOR,TOTAL_GHG: MEAN_INTENSITY,CARBON_INTENSITY
723,2020,CRH Finland Services OYJ,33,2.07,Other sectors,27200000.0,13140096.62
983,2021,CRH Finland Services OYJ,51,2.07,Other sectors,27200000.0,13140096.62
1256,2022,CRH Finland Services OYJ,8,2.07,Other sectors,27200000.0,13140096.62
565,2019,Holcim Finance S.A.,208,103.91,Other sectors,163219755.0,1570780.05
1066,2021,Holcim Finance S.A.,480,103.91,Other sectors,163219755.0,1570780.05


In [16]:
carbon_intensity_companies.to_csv(path + 'Carbon_Intensity_Companies.csv',index = None)

### Bonds distribution per sector

In [17]:
# number of bonds in each sector
cols = ['YEAR','ECONOMIC_SECTOR','ISIN_CODE']
file2 = master_file[cols]

df2 = file2.groupby(['YEAR','ECONOMIC_SECTOR']).count()
df2 = df2.reset_index()
columns = ['YEAR','ECONOMIC_SECTOR','NUMBER_OF_BONDS']
df2.columns = columns
df2.head()

,YEAR,ECONOMIC_SECTOR,NUMBER_OF_BONDS
0,2017,Automotive and parts,28
1,2017,Beverages,287
2,2017,Chemicals,571
3,2017,Construction & Materials,847
4,2017,Energy and basic resources,1886


In [18]:
df2.to_csv(path + 'Bonds_sectorwise.csv',index=None)

### Bonds distribution per country

In [19]:
file3 = risk_country(master_file)

In [20]:
cols = ['YEAR','RISK_COUNTRY','ISIN_CODE']
file3 = file3[cols]

df3 = file3.groupby(['YEAR','RISK_COUNTRY']).count()
df3 = df3.reset_index()
columns = ['YEAR','RISK_COUNTRY','NUMBER_OF_BONDS']
df3.columns = columns
df3.head()


,YEAR,RISK_COUNTRY,NUMBER_OF_BONDS
0,2017,Belgium,1089
1,2017,France,7394
2,2017,Germany,2803
3,2017,Italy,3123
4,2017,Netherlands,6097


In [21]:
df3.to_csv(path + 'Bonds_countrywise.csv',index = None)